In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, desc

In [9]:
spark = SparkSession.builder.appName("TrendingSongsAnalysis").getOrCreate()

In [10]:
df = spark.read.csv("dataset_music_1.csv", header=True, inferSchema=True)

In [11]:
df.show(5)

+---+--------------------+--------------------+--------------------+--------------------+----------+-----------+--------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+--------------+-----------+--------+----------+
|_c0|            track_id|             artists|          album_name|          track_name|popularity|duration_ms|explicit|danceability|energy|key|loudness|mode|speechiness|acousticness|instrumentalness|liveness|valence|  tempo|time_signature|track_genre| User ID|play_count|
+---+--------------------+--------------------+--------------------+--------------------+----------+-----------+--------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+--------------+-----------+--------+----------+
|  0|5SuOikwiRyPMVoIQD...|         Gen Hoshino|              Comedy|              Comedy|        73|     230666|   FALSE|       0.676| 0.461|1.0|  -6.746| 0.0|      0.143|      0

In [12]:
df_clean = df.filter(df["play_count"].isNotNull() & df["artists"].isNotNull())

In [13]:
df_clean = df_clean.withColumn("play_count", col("play_count").cast("int"))

In [14]:
genre_artist_agg = df_clean.groupBy("track_genre", "artists") \
                           .agg(sum("play_count").alias("total_play_count")) \
                           .orderBy(desc("total_play_count"))

In [15]:
genre_artist_agg.show(10)

+-----------+--------------------+----------------+
|track_genre|             artists|total_play_count|
+-----------+--------------------+----------------+
|          4|    Rachael Yamagata|        15792102|
|      blues|     Ella Fitzgerald|             132|
|      anime|                Kato|              69|
|  bluegrass|  Greensky Bluegrass|              67|
|   afrobeat|              Criolo|              60|
|   afrobeat|Los Amigos Invisi...|              60|
|      blues|               Slank|              58|
|    ambient|    Ludovico Einaudi|              56|
|   afrobeat|     Plastilina Mosh|              52|
|   afrobeat|       Jorge Drexler|              52|
+-----------+--------------------+----------------+
only showing top 10 rows

